<a href="https://colab.research.google.com/github/MariaPosadaBotero/Entregas-del-proyecto-de-AI-para-ingenieros---Udea-2025-2/blob/main/03-modelo%20con%20preprocesado%20SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MODELO SVM

In [1]:
#IMPORTAMOS LAS LIBRERIAS NECESARIAS
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report


In [2]:
#CARGAR LOS CSV PREPROCESADOS
train = pd.read_csv("train_preprocesado.csv")
test  = pd.read_csv("test_preprocesado.csv")

print(train.shape, test.shape)



/tmp/ipython-input-1160106152.py:3: DtypeWarning: Columns (804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023) have mixed types. Specify dtype opti

(692500, 21) (235740, 1024)


In [4]:
# VALIDAR QUE EXISTA LA COLUMNA TARJET
print(train.columns)

if "target" not in train.columns:
    raise ValueError("ERROR: El CSV de train NO tiene la columna 'target'.")


Index(['PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4', 'target'],
      dtype='object')


In [5]:
#REDUCIR LA MEMORIA DE LOS ARCHIVOS PREPROCESADOS
def reducir_memoria(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type == "int64":
            df[col] = df[col].astype("int32")

        elif col_type == "float64":
            df[col] = df[col].astype("float32")

        elif col_type == "object":
            # Convertir textos a category para ahorrar memoria
            df[col] = df[col].astype("category")

    return df

train = reducir_memoria(train)
test  = reducir_memoria(test)


In [6]:
# SEPARAR X_TRAIN y Y_TRAIN
X_train = train.drop(columns=["target"])
y_train = train["target"]

X_test = test.copy()

print("Columnas de X_train:", X_train.shape)


# CONSERVAR SOLO COLUMNAS NUMERIAS
X_train = X_train.select_dtypes(include=["int32","float32"])
X_test  = X_test.select_dtypes(include=["int32","float32"])

print("Columnas numéricas:", X_train.shape[1])


Columnas de X_train: (692500, 20)
Columnas numéricas: 5


In [7]:
# EVALUAR EL MODELO CON EL ACCURACY
from sklearn.model_selection import cross_val_score


In [8]:
# Usamos el mismo modelo que entrenas después
modelo_eval = LinearSVC(
    C=1.0,
    max_iter=5000,
    dual=False
)

scores = cross_val_score(modelo_eval, X_train, y_train, cv=5, scoring="accuracy")

print("Resultados cross-validation:", scores)
print("Accuracy promedio:", scores.mean())

Resultados cross-validation: [0.33051986 0.32958845 0.33197834 0.32847653 0.33024549]
Accuracy promedio: 0.3301617328519856


In [9]:
# ENTRENAR EL MODELO SVM LIVIANO
modelo = LinearSVC(
    C=1.0,
    max_iter=5000,
    dual=False
)

modelo.fit(X_train, y_train)
print("Modelo entrenado correctamente.")


Modelo entrenado correctamente.


In [10]:
# CREAR PREDICCIONES
predicciones = modelo.predict(X_test)
predicciones[:10]


array([3, 1, 3, 0, 0, 0, 3, 3, 0, 1], dtype=int32)

In [11]:
# GUARDAR PREDICCIONES
pd.DataFrame({"pred": predicciones}).to_csv("predicciones_svm.csv", index=False)
print("Archivo predicciones_svm.csv guardado.")

predicciones = modelo.predict(X_test)


Archivo predicciones_svm.csv guardado.


In [12]:
# CREAR ARCHIVO DE SUBMISSION PARA KAGGLE
import numpy as np
import pandas as pd

# Crear IDs consecutivos (Kaggle los necesita)
ids = np.arange(len(predicciones))

submission = pd.DataFrame({
    "id": ids,
    "target": predicciones
})

submission.to_csv("submission.csv", index=False)

print("Archivo 'submission.csv' creado correctamente.")
submission.head()

Archivo 'submission.csv' creado correctamente.


,id,target
0,0,3
1,1,1
2,2,3
3,3,0
4,4,0
